In [9]:
pip install scipy

You should consider upgrading via the '/Users/anaistang/.virtualenvs/web/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import requests 
import pandas as pd 
import requests 
import math 
from scipy import stats
import xlsxwriter

In [11]:
from starter.secrets import IEX_CLOUD_API_TOKEN
stocks = pd.read_csv('starter/sp_500_stocks.csv')


## Getting one-year price returns for each stock in the universe

In [12]:
symbol = 'aapl'
url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
r = requests.get(url)
data = r.json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2243423019779,
 'week52high': 147.32,
 'week52low': 90.9,
 'week52highSplitAdjustOnly': 146.96,
 'week52lowSplitAdjustOnly': 91.81,
 'week52change': 0.509370203019638,
 'sharesOutstanding': 17057152563,
 'float': 0,
 'avg10Volume': 83847207,
 'avg30Volume': 76709925,
 'day200MovingAvg': 133.5,
 'day50MovingAvg': 128.17,
 'employees': 154136,
 'ttmEPS': 4.56,
 'ttmDividendRate': 0.8338776784125632,
 'dividendYield': 0.00654272340733022,
 'nextDividendDate': '',
 'exDividendDate': '2021-04-30',
 'nextEarningsDate': '2021-07-21',
 'peRatio': 29.239712153511867,
 'beta': 1.536590361972954,
 'maxChangePercent': 49.34395723229179,
 'year5ChangePercent': 4.883174155493546,
 'year2ChangePercent': 1.761148614121663,
 'year1ChangePercent': 0.5087761389243302,
 'ytdChangePercent': -0.014014423133404463,
 'month6ChangePercent': 0.03333498074458751,
 'month3ChangePercent': 0.08732187786523074,
 'month1ChangePercent': 0.04579808469510678,
 'day30ChangePerce

## Parsing API call


In [13]:
change_percent = data['year1ChangePercent']
change_percent

0.5087761389243302

In [14]:
def chunks(lst, n):
    """
    Splitting list into evenly sized chunks
    """
    for i in range(0, len(lst), n):
        yield lst[i:i + n]



In [15]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
symbol_strings

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA',
 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD',
 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,I

In [16]:
columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_df = pd.DataFrame(columns=columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
#     print(data['A'])
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
        pd.Series(
            [
                symbol, 
                data[symbol]['stats']['week52high'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
            ],
            index=columns),
            ignore_index=True
            )
final_df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,150.88,0.665513,N/A
1,AAL,26.71,0.367941,N/A
2,AAP,214.52,0.328677,N/A
3,AAPL,146.99,0.507196,N/A
4,ABBV,123.04,0.240019,N/A
...,...,...,...,...
95,CINF,126.75,0.884793,N/A
96,CL,86.11,0.130956,N/A
97,CLX,239.00,-0.185348,N/A
98,CMA,79.48,0.793186,N/A


## Removing Low-Momentum Stocks
- We're identifying 50 highest-momentum stocks in the S&P 500
- We need to remove all the stocks in our DataFrame that fall below this momentum threshold.
- Sort the Dataframe by the stocks one-year price return, drop all stocks outside the top 50.

In [17]:
final_df.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_df = final_df[:50]
final_df.reset_index(inplace=True) # sets first row as first index 
final_df 

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,24,ALGN,664.10,1.326636,N/A
1,29,AMAT,151.35,1.244759,N/A
2,78,CARR,49.22,1.157221,N/A
3,23,ALB,190.30,1.041435,N/A
4,45,APTV,166.53,0.955624,N/A
5,16,AES,28.95,0.939777,N/A
6,95,CINF,126.75,0.884793,N/A
7,82,CBRE,91.43,0.835543,N/A
8,51,AVY,230.14,0.817090,N/A
9,90,CFG,52.66,0.800257,N/A


In [18]:
## Calculating the number of shares to buy 

In [22]:
def portfolio_size():
    global portfolio_size 
    portfolio_size = input('Enter the size of your portfolio:\n')
    
    try:
        float(portfolio_size)
    except:
        print('This is not a number \n Please try again:')
        portfolio_size = input('Enter the size of your portfolio')
        
portfolio_size()
print(portfolio_size)
        

Enter the size of your portfolio:
10000000
10000000


In [29]:
position_size = float(portfolio_size)/len(final_df.index)
# print(position_size)
for i in range(0, len(final_df)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_df.loc[i, 'Price'])
final_df

/Users/anaistang/.virtualenvs/web/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,24,ALGN,664.10,1.326636,301
1,29,AMAT,151.35,1.244759,1321
2,78,CARR,49.22,1.157221,4063
3,23,ALB,190.30,1.041435,1050
4,45,APTV,166.53,0.955624,1200
5,16,AES,28.95,0.939777,6908
6,95,CINF,126.75,0.884793,1577
7,82,CBRE,91.43,0.835543,2187
8,51,AVY,230.14,0.817090,869
9,90,CFG,52.66,0.800257,3797


In [ ]:
## Building a realistic momentum strategy 
- High-quality momentum stocks show 'slow and steady' outperformance over long periods of time 
- Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards

High-quality momentum stocks are preferred because low-quality momentum can often be caused by short